# TRABAJO PRÁCTICO FINAL - ESTRUCTURAS DE DATOS
### GRUPO: Yarará

#### Integrantes:
- Eduardo Reichel
- Liber Rosetti
- Nicolás Racedo
- Matías Correa

#### Enlace al video explicativo:
[TP YARARÁ](https://www.youtube.com/watch?v=8ipaxdeOUaU&ab_channel=EduardoEmanuelReichel)

## INFORME

En el contexto de la materIa Estructuras de datos de la carrera Ingeniería de Computación, se presenta el trabajo integrador de la misma.
El trabajo consiste en un programa diseñado en Python que compara los precios de productos de distintas páginas y los devuelve en un archivo de formato de salida específico.

La estructura del informe esta organizado en análisis de las páginas selecionadas, diagramas de clase y decisiones de diseño. 

Para poder ejectuar el programa, es necesario que el usuario tenga instalada ciertas librerias, que se detallan a continuación:
- SCRAPY
- NLTK (se debe descargar dos paquetes que el programa te indica)
- JSONPICKLE

Se destaca que el programa fue testeado en distintos sistemas operativos:
- Windows 10
- Linux Ubuntu 20.04 LTS

### DIAGRAMAS DE CLASE

En la siguiente imagen se puede observar el diagrama de clases del proyecto:

<img src="uml.png">

### DECISIONES DE DISEÑO

A continuación se explican las decisiones de diseño que se tomaron en cuenta para el trabajo.
En primer lugar, al elegir las páginas para analizar, se decidió escrapear toda la página entera, por categorías de productos. Por esto nos referimos a que al ejecutar el programa, el mismo va página por página analizando cada una de las categorías y cada uno de los productos. Esto fue porque no todas las páginas tenian URL con Jquery para realizar la búsqueda de productos y por otro lado, buscamos que todas las páginas tengan una misma estructura para no generar técnicas de escrapeo distintas.

Con respecto al menú, esta diseñado para que el usuario avance por las distintas opciones ingresando números que hacen referencia a las distintas opciones. Se aclara que al ejecutar el programa, si el usuario ingresa el número "1", el programa llega a su fin.

En los spiders, se utilizó la clase Items con el objetivo de organizar el formato de salida de los elementos de buscados.

Se decidió que la información de salida (los archivos que se pueden obtener Json, HTML, CSV) se almacenan en una carpeta del proyecto que se llama "Resultados". Se toma el path desde la dirección donde se está corriendo el programa y en caso de que no esté creada la carpeta "Resultados", la crea.

Como archivo de configuración, se creó el archivo "config.py". En el mismo se detallan las páginas, formatos admitidos de salida, tipos de busqueda, las start URL y los métodos get de cada uno de los tipos.

Se decidió utilizar CrawlerProcess para poder corres los spiders. Con esta herramienta se logró poder ejecutar cada uno de los spiders en paralelo. Durante el desarrollo, se intentó implementar Reactor, pero al utilizarlo se encontraron inconvenientes durante la ejecución y se optó por otra opción.

Para los items opcionales, se tuvieron dos consideraciones:

- Por un lado, en caso de que los productos no tuviesen precio, se decidió que el precio del producto va a ser igual a cero.
- Por otro lado, en cuanto al item que solicita ordenar los productos de una busqueda que ingrese el usuario, nos encontramos con el problema de que un mismo producto; en distintas páginas, puede tener un nombre distinto. 

Ejemplo:
   - **Celular Libre Samsung Galaxy S20 Ultra Gris** (Fravega)
   - **Celular Libre Samsung S20 6.2" 8/128 Gris** (Casa Del Audio)
   - **CELULAR LIBRE SAMSUNG S20 ULTRA GRIS** (Cetrogar)

Para poder solucionar esto, se implementó la librería difflib. Esta librería analiza el porcentaje de exactitud entre distintas frases. Es necesario setear un escalar que me define el nivel de exactitud que quiero obtener. Cero significa que no se quiere exactitud, 1 significa que la búsqueda tiene que ser completamente exacta. Optamos tomar por un valor de 0.8 en base a pruebas empíricas. Se probó tomar un valor de 1 pero ninguna de las páginas tiene el mismo nombre en cada producto.

Para la página de Musimundo, nos encontramos que al buscar los precios de los productos, la página devolvía dos precios distintos dependiendo de donde lo busque, como se ve en la siguiente imagen:

<img src="musimundo_precios.png">


Decidimos en este caso, quedarnos con el de menor valor, ya que era del mismo lugar de donde recuperamos los precios de los demás productos.

Por último, se encontró que a lo largo del código se repetían varías secciones en la lógica utilizada por los spider para definir, según los tipos de búsqueda, si es válido el producto encontrado. Para eso, se implemento una clase "utils.py".

### ANÁLISIS DE LAS PÁGINAS

Se decidió trabajar con páginas de electrodomésticos que funcionen a nivel nacional. El primer listado de páginas que se pensó fue el siguiente:

- [Frávega](https://www.fravega.com/)
- [Musimundo](https://www.musimundo.com/)
- [Cetrogar](https://www.cetrogar.com.ar/)
- [Rodo](https://www.rodo.com.ar/)
- [Casa Del Audio](https://www.casadelaudio.com/)
- [Carrefour](https://www.carrefour.com.ar/)
- [Falabella](https://www.falabella.com.ar/falabella-ar/)
- [Garbarino](https://www.garbarino.com)
- [Grupo Marquez](https://grupomarquez.com.ar/)


En este punto, lo primero que se analizó fue si cada unas de estas páginas podía ser scrapeada. 
Para eso, se ejecutaron los siguientes comandos por consola para analizar lo antes mencionado:
    
    scrapy shell "URL"
    
En los casos de páginas que nos retornan por consola la siguiente linea:

   [scrapy.core.engine] DEBUG: Crawled (200) <GET URL>
    
interpretamos que vamos a poder scrapear la página y traernos la información que deseamos.
Se encontró dos inconvenientes a la hora de seleccionar las páginas. 
Por un lado, por ejemplo en la página de Rodo y Garbarino, al ejecutar el siguiente comando
    scrapy shell "www.rodo.com.ar"

Se obtuvo lo siguiente:

[scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (302) to <GET https://rodo.com.ar/> from <GET https://www.rodo.com.ar/productos.html>

Esto nos indica que la página no puede ser scrapeada.
Otra página con la que tuvimos problema fue garbarino, que al querer analizar si se puede scrapear nos devolvió la siguientes lineas:

    [scrapy.core.engine] DEBUG: Crawled (403) <GET https://www.garbarino.com:443/> (referer: None)
    [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.garbarino.com:443/> from <GET http://www.garbarino.com>
    
Por otro lado, ciertas páginas estan realizadas con frameworks relativamente nuevos (EJ: React), que hace que la página sea dinámica y no se pueda acceder a todos los productos de una vez. En esos casos, se descartaron las páginas.

Las 4 páginas que finalmente seleccionamos para el trabajo fueron:

   - [Frávega](https://www.fravega.com/)
   - [Cetrogar](https://www.cetrogar.com.ar/)
   - [Casa Del Audio](https://www.casadelaudio.com/)
   - [Musimundo](https://www.musimundo.com/)
  
Como aclaramos anteriormente, lo que hicimos fue buscar páginas que se recorrieran de la misma manera. Es decir, páginas que tuvieran un elemento que tuviera cada una de las categorías que ofrecian y de cada categoría recorer cada uno de los productos.
En la siguiente imagen se puede observar un ejemplo con cetrogar


<img src="cetrogar_categorias.png">


El recorrido de la página se analizó utilizando xpath y buscando los elementos que van desde el inicio de la página hasta el listado de productos. En la imagen que sigue se puede ver un ejemplo:


<img src="cetrogar_xpath.png">

Por último, el inconveniente que se presentaba era como acceder a los productos que se encuentren en otras páginas.
En todas las que utilizamos, encontramos que tiene un boton de "next" el cual está asociado a un link mediante la propiedad href. 

De cada uno de los productos, lo que recuperamos fue el precio, el nombre completo, la categoría del producto y la fecha en la que se busca el producto.

In [ ]:
pip install jsonpickle

In [ ]:
from menu import Menu

In [ ]:
menu = Menu()

In [ ]:
menu.ejecutar()

In [ ]:
Los archivos han sido creados en una carpeta llamada "Resultados" en el path donde se está ejecutando este documento.

In [ ]:
pip install -U ipytest

In [ ]:
import ipytest
ipytest.autoconfig()

In [ ]:
%%run_pytest[clean]

from utils import Utils
from procesador import Procesador
from os import path
import pathlib
import datetime
from config import Config
import menu
from menu import QuiereSalirException
import unittest
from unittest.mock import patch

utils = Utils()
def test_tipo_busqueda_1():
    '''Publicación con la frase exacta'''
    target = 'Lavarropas Carga Frontal Longvie 8 Kg 1200 RPM L18012'
    title = 'Lavarropas Carga Frontal Longvie 8 Kg 1200 RPM L18012'
    assert utils.tipo_busqueda_1(target, title)

def test_tipo_busqueda_2():
    '''Publicación que contenga todas las palabras'''
    target = 'Lavarropas Carga Frontal Longvie 8 Kg 1200 RPM L18012'
    title = 'Lavarropas Carga Frontal Longvie 8 Kg 1200 RPM L18012'
    assert utils.tipo_busqueda_2(target, title)

def test_tipo_busqueda_2_v1():
    '''Publicación que contenga todas las palabras'''
    target = 'Lavarropas CARGA Frontal'
    title = 'Lavarropas Carga Frontal Longvie 8 Kg 1200 RPM L18012'
    assert utils.tipo_busqueda_2(target, title)

def test_tipo_busqueda_3():
    '''Publicación que contenga algunas de las palabras.'''
    target = 'Lavarropas Longvie'
    title = 'Lavarropas Carga Frontal Longvie 8 Kg 1200 RPM L18012'
    assert utils.tipo_busqueda_3(target, title)

def test_tipo_busqueda_3_v1():
    '''Publicación que contenga todas las palabras'''
    target = 'Celular Samsung'
    title = 'CELULAR LIBRE SAMSUNG S20 ULTRA GRIS'
    assert utils.tipo_busqueda_3(target, title)

#fragmeto extraido de un scrapeo donde criterio fue longvie, pero para lo cambio a restados_test para test
lista = [{'categoria': 'Electrodomésticos','fecha': '15/11/2020 12:26:02',
 'link': 'https://www.cetrogar.com.ar/aspiradora-electrolux-gt30n-1300w.html',
 'market': 'cetrogar', 'price': 12109.0,
 'title': 'Calefactor Longvie EC3200ECA3 Con Multigas'}, {'categoria': 'Electrodomésticos',
 'fecha': '15/11/2020 12:26:07',
 'link': 'https://www.cetrogar.com.ar/aspiradora-yelmo-as-3228-2000w-sin-bolsa.html',
 'market': 'cetrogar', 'price': 23899.0,
 'title': 'Termotanque Longvie Electrico para colgar TE40F Carga Inferior'}, {'categoria': 'Electrodomésticos',
 'fecha': '15/11/2020 12:26:07',
 'link': 'https://www.cetrogar.com.ar/cafetera-nespressor-inissia-red-aeroccino-3-a3c40-ar-re-ne.html',
 'market': 'cetrogar', 'price': 29999.0,
 'title': 'Termotanque Longvie Multigas Colgar 50 Litros inferior'}, {'categoria': 'Electrodomésticos',
 'fecha': '15/11/2020 12:26:08',
 'link': 'https://www.cetrogar.com.ar/microondas-bgh-qchef-28l-b228db-bl.html',
 'market': 'cetrogar', 'price': 43799.0,
 'title': 'Lavarropas Longvie L16508 6.5Kg 800rpm'}, {'categoria': 'Electrodomésticos',
 'fecha': '15/11/2020 12:26:08',
 'link': 'https://www.cetrogar.com.ar/anafe-longvie-a-6600g-x-4-hornallas-gas.html',
 'market': 'cetrogar', 'price': 48999.0, 'title': 'Anafe Longvie A-6600G x 4 Hornallas Gas'}]

criterio = 'resultados test'.replace(' ','_')
procesador = Procesador(lista, criterio)

def test_imprimir_archivo_csv():
    procesador.imprimirArchivo('csv')
    filename = criterio + "_" + datetime.datetime.now().strftime("%d-%m-%Y") + '.csv'
    assert path.isfile('resultados/' + filename)

def test_imprimir_archivo_json():
    procesador.imprimirArchivo('json')
    filename = criterio + "_" + datetime.datetime.now().strftime("%d-%m-%Y") + '.json'
    assert path.isfile('resultados/' + filename)

def test_imprimir_archivo_html():
    procesador.imprimirArchivo('html')
    filename = criterio + "_" + datetime.datetime.now().strftime("%d-%m-%Y") + '.html'
    assert path.isfile('resultados/' + filename)

config = Config()
def test_config_paginas():
    assert config.get_paginas()['5'] == 'Todas'

def test_config_formatos_admitidos():
    assert config.get_formatos_admitidos()['2'] == 'json'

def test_config_tipos():
    assert config.get_tipos()['3'] == 'Alguna de las palabras'

def test_config_url():
    assert config.get_start_url()['fravega'] == 'https://www.fravega.com/'

def test_config_extras():
    assert config.get_extras()['1'] == 'Estadísticas por modelo'

#Test de Menu
class TestMenu(unittest.TestCase):
    
    menu_t = menu.Menu()
    
    # Test para formato '1'
    @patch("builtins.input", return_value="1")
    def test_pedir_formato_1(self, mock_input):
        self.assertEqual(self.menu_t._Menu__pedir_formato(), "1")
        
    # Test para input '0' (Quiere salir)
    @patch("builtins.input", return_value="0")
    def test_pedir_formato_0(self, mock_input):
        with self.assertRaises(QuiereSalirException):
            self.menu_t._Menu__pedir_formato()
            
    # Test para formato pedir pagina
    @patch("builtins.input", return_value="3")
    def test_pedir_pagina_3(self, mock_input):
        self.assertEqual(self.menu_t._Menu__pedir_pagina(), ["3"])
        
    # Test para formato pedir varias paginas
    @patch("builtins.input", return_value="3,1,2")
    def test_pedir_paginas(self, mock_input):
        self.assertEqual(self.menu_t._Menu__pedir_pagina(), ["3", '1', '2'])
        
    # Test para input '0' (Quiere salir)
    @patch("builtins.input", return_value="0")
    def test_pedir_pagina_0(self, mock_input):
        with self.assertRaises(QuiereSalirException):
            self.menu_t._Menu__pedir_pagina()
    
    # Test para formato pedir tipo busqueda
    @patch("builtins.input", return_value="2")
    def test_pedir_tipo_busqueda(self, mock_input):
        self.assertEqual(self.menu_t._Menu__tipo_busqueda(), '2')
        
    # Test para input '0' (Quiere salir)
    @patch("builtins.input", return_value="0")
    def test_pedir_tipo_busqueda_0(self, mock_input):
        with self.assertRaises(QuiereSalirException):
            self.menu_t._Menu__tipo_busqueda()
            
    # Test para formato pedir extras
    @patch("builtins.input", return_value="2")
    def test_pedir_extra_2(self, mock_input):
        self.assertEqual(self.menu_t._Menu__pedir_extras(), '2')
        
    # Test para input '0' (Quiere salir)
    @patch("builtins.input", return_value="0")
    def test_pedir_extra_0(self, mock_input):
        with self.assertRaises(QuiereSalirException):
            self.menu_t._Menu__pedir_extras()
